In [ ]:
# MODEL = 'efficientdet-d0' \
# import os \
# def download(m):\
#   if m not in os.listdir():\
#     !wget https://storage.googleapis.com/cloud-tpu-checkpoints/efficientdet/coco/{m}.tar.gz \
#     !tar zxf {m}.tar.gz \
#   ckpt_path = os.path.join(os.getcwd(), m) \
#   return ckpt_path

In [1]:
import wandb

In [2]:
run = wandb.init(project="test_project", name="Test Run")

wandb: Currently logged in as: asadrevolveai (use `wandb login --relogin` to force relogin)


In [3]:
import tensorflow as tf

In [4]:
model = tf.keras.applications.mobilenet.MobileNet(
    input_shape=None, alpha=1.0, depth_multiplier=1, dropout=0.001,
    include_top=True, weights='imagenet', input_tensor=None, pooling=None,
    classes=1000, classifier_activation='softmax'
)

2021-07-17 14:00:51.432500: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2021-07-17 14:00:51.432759: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2021-07-17 14:00:51.434627: I tensorflow/core/common_runtime/process_util.cc:146] Creating new thread pool with default inter op setting: 2. Tune using inter_op_parallelism_threads for best performance.


In [29]:
tf.saved_model.load('./package/models/saved_models/efficientdet-d0')

OSError: SavedModel file does not exist at: ./package/models/saved_models/efficientdet-d0/{saved_model.pbtxt|saved_model.pb}

In [25]:
model = tf.keras.models.load_model('./package/models/saved_models/efficientdet-d0.h5')

ValueError: No model found in config file.

wandb: Network error (ConnectionError), entering retry loop.
wandb: Network error resolved after 0:00:09.208540, resuming normal operation.


In [5]:
%load_ext autoreload
%autoreload 2
import package as pk

In [6]:
benchmarker = pk.utils.Benchmark(model=model, batch_size=4, img_size=(224, 224))

In [7]:
benchmarks = benchmarker.execute()

Testing on CPU


2021-07-17 14:00:57.561363: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2021-07-17 14:00:57.579639: I tensorflow/core/platform/profile_utils/cpu_utils.cc:112] CPU Frequency: 2194880000 Hz



        Inference Time (milliseconds):   322.59
        Throughput:                     80.65
        Standard Deviation of Time:     37.05
        


In [9]:
iters = benchmarks.pop('inference_time_iters')

In [13]:
import numpy as np

[[1, 297.8052080143243],
 [2, 306.21123098535463],
 [3, 339.05474899802357],
 [4, 322.9088560328819],
 [5, 374.5409219991416],
 [6, 393.23652803432196],
 [7, 316.3642860017717],
 [8, 255.23558596614748],
 [9, 312.14303901651874],
 [10, 308.4252669941634]]

In [17]:
table = wandb.Table(data=[[x,y] for (x, y) in zip(np.arange(len(iters))+1, iters)], columns=["Iterations", "Inference time (milliseconds)"])

In [18]:
wandb.log({"infer_time_plot" : wandb.plot.line(table, "Iterations", "Inference time (milliseconds)",
           title="Inference time in each iterations")})

In [20]:
wandb.log(benchmarks)

In [17]:
# run.join()